# What factors contribute to customer satisfaction? 

1.  Explorer the data 
2.  Label Encording
3.  Data Visualisation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
air_plane_train = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
air_plane_test = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

air_plane_train.head()

In [ ]:
air_plane_test = air_plane_test.dropna()

In [ ]:
#null value check
air_plane_train = air_plane_train.dropna()

In [ ]:
air_plane_test.isna().sum()

Both train and test data has a null values in in the Arrival Delay in Minutes. I am not going to drop null in this level. I will drop or imputed null values after the checking correlations.

There are some catagarical data which aren't in numeric format. We should use label encording to encorded the catagarical data into numerical data.

In [ ]:
catagrycal_columns = air_plane_train.loc[:,air_plane_train.dtypes == object ].columns

catagrycal_columns

In [ ]:
#we should remove id, dropidex,
air_plane_train = air_plane_train.drop(['id', 'Unnamed: 0'], axis=1)
air_plane_test = air_plane_test.drop(['id', 'Unnamed: 0'], axis=1)
#get copy of the original dataste


In [ ]:

#get copy of the original dataste
encoded_plane_data_train = air_plane_train.copy()
encoded_plane_data_test = air_plane_test.copy()

label_encoder = LabelEncoder()

for col in catagrycal_columns:
    encoded_plane_data_train[col] = label_encoder.fit_transform(air_plane_train[col])
    encoded_plane_data_test[col] =  label_encoder.fit_transform(air_plane_test[col]) 


In [ ]:
encoded_plane_data_train.describe()


In [ ]:
plt.figure(figsize=(20,10))
plt.title('The satisfaction correlation')
sns.heatmap(encoded_plane_data_train.corr(), annot=True, fmt='.0%')


There are some features not correlated with the satisfaction.So, I am going to drop all of them from the dataset, and keep only positive correlated features

In [ ]:
encoded_plane_data_train = encoded_plane_data_train.drop(['Customer Type', 
                                                          'Type of Travel', 
                                                          'Class', 
                                                          'Departure/Arrival time convenient', 'Gate location',
                                                          'Departure Delay in Minutes', 'Arrival Delay in Minutes'], axis=1)
encoded_plane_data_test = encoded_plane_data_test.drop(['Customer Type', 
                                                          'Type of Travel', 
                                                          'Class', 
                                                          'Departure/Arrival time convenient', 'Gate location',
                                                          'Departure Delay in Minutes', 'Arrival Delay in Minutes'], axis=1)

In [ ]:
sns.pairplot(encoded_plane_data_train)


split data to feature and target

In [ ]:
#split data X,y
X = encoded_plane_data_train.iloc[:, :-1]
y = encoded_plane_data_train.iloc[:,-1:]

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X,y, test_size=0.2)


In [ ]:
model = RandomForestClassifier(max_features = 3, n_estimators=100, random_state = 0)
model.fit(X_train, y_train)
preds = model.predict(X_test)

#mae = mean_absolute_error(y_test, preds)
# train_fit_score = model.score(X_train, y_train)
# test_fit_score = model.score(X_test, y_test)
# accuracy_score = accuracy_score(y_test, preds)

# print('mean_absolute_error: {}'.format(mae))
# print('train_fit_score: {}'.format(train_fit_score))
# print('test_fit_score: {}'.format(test_fit_score))
# print('accuracy_score: {}'.format(accuracy_score))

X_test

In [ ]:
y_test

In [ ]:
 plus = model.predict(encoded_plane_data_test.iloc[:, :-1])
    

In [ ]:
test = encoded_plane_data_test.iloc[:,-1:].head(10)
test

In [ ]:
accuracy_score(encoded_plane_data_test.iloc[:, -1:], plus)

In [ ]:
#classification_report for the test data test
reportLReg = classification_report(encoded_plane_data_test.iloc[:, -1:], plus, output_dict=True)
pd.DataFrame(reportLReg).transpose()

In [ ]:
roc_auc_score(encoded_plane_data_test.iloc[:, -1:],plus)

In [ ]:
test_data = encoded_plane_data_test.iloc[:, :-1]

test_data['satisfaction'] = label_encoder.inverse_transform(plus)

In [ ]:
#model predicted data
test_data.head(7)

In [ ]:
#given dataset to test
air_plane_test.dropna().head(7)

In [ ]:
from sklearn.pipeline import Pipeline

my_pipeline = Pipeline(steps=[
                              ('model', RandomForestClassifier(max_features = 3, n_estimators=100, random_state = 0))
                             ])

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

In [ ]:
print("Average MAE score (across experiments):")
print(scores.mean())